Before we start, run the code cell below for a nicer layout.

In [2]:
%%html
<style>
h1 { margin-top: 3em !important; }
h2 { margin-top: 2em !important; }
h3 { margin-top: 1em !important; }
#notebook-container { 
    width: 50% !important; 
    min-width: 800px;
}
</style>

<h1>The shark dataset</h1>

<br/>

<div style="border: 4px solid #f00; padding: 5px;">
    The dataset contains graphic descriptions of injuries. You 
    will <b>not</b> have to work with those in any way, but please
    be aware if you decide to have a look at the file in a text
    editor.
</div>

We have already seen the shark attack dataset in the lecture, it 
is contained in the csv file `04-resources/shark-attacks.csv` that 
came with this notebook. I download the file as it is from a famous dataset collection and challenge website called <a href="http://www.kaggle.com/">Kaggle</a>. Today we will see how to import such a dataset using pandas.

Yes, I agree that we have done pandas imports in a previous lab, but today
we will not focus so much on the act of getting the .csv file into
a pandas Dataframe (of course, repeating this procedure is a good
learning exercise) but on the steps <i>afterwards</i>. Before we can set about exploring any dataset, we first need to make sure that it is in good form!

> Load the shark attack dataset using the method `.read_csv(...)`
  of the `pandas` package. Keep the variable name `sharks_raw`.
  
  


In [ ]:
import pandas as pd

sharks_raw = ... # TODO
sharks_raw # Display 

Take a moment to scroll through the dataset (or open the file `04-resources/shark-attacks.csv` in your favourite text editor). You
should notice a few issues with the data, among them:
<ol>
    <li> A lot of the columns are not interesting to us (<tt>href</tt>, <tt>Case Number.1</tt>)
    <li> Many fields are missing, in particular towards the end 
         of the file
    <li> The <tt>year</tt> column uses the value 0 to denote an unknown value, but sometimes it is simply missing
</ol>

Let us create a &lsquo;clean&rsquo; dataset `sharks` that removes
those issues. First, let us select a subset of the data:

In [ ]:
sharks = sharks_raw[['Year', 'Country', 'Activity', 'Age', 'Species ']].copy()
# The columns 'Sex ' and 'Species ' have a space at the end,
# so we copy them into differently named columns in the new dataset
sharks['Sex'] = sharks_raw['Sex '].copy()
sharks['Species'] = sharks_raw['Species '].copy()
sharks.head(5) # Display first few rows

<h2>Cleanup</h2>

<h3>Removing rows with missing values</h3>

We want to remove all rows from the dataframe that contain
a NaN (&ldquo;Not a Number&rdquo;) value. Our first reflex might
be to iterate through the rows, check whether they contain a NaN
value and re-build the dataframe with those rows that don't.

This is, however, not in the spirit of the `pandas` module and quite 
inefficient. We will learn a different method now which is more in 
line of how `pandas` is supposed to be used. Let us create a small dataset to test it out.

In [ ]:
minisharks = sharks.head(5)
minisharks

`pandas` dataframes allow us to select rows by giving it a list of booleans. For example, the following code selects the first, second, and forth row (try changing the boolean values to see how it affects the result):

In [ ]:
selection = [True,True,False,True,False]
minisharks[selection]

The idea is now that we construct a boolean list which indicates for every row whether it contains a NaN value or not. Dataframes come with a method to test whether a field is NaN or not:

In [ ]:
minisharks.isna()

Notice how NaN values are replaced by `True` while all other values are replaced by `False`. Now, to see whether a given row has a NaN value or not, we want to take the logical or over all these values; e.g. we want the result to be `True` if one or more fields of the row is `True` and only `False` if <i>all</i> fields are `False`.

Python has the basic functions `any(...)` and `all(...)` to do these kind of logical (or &lsquo;Boolean&rsquo;, named after British mathematician <a href="https://en.wikipedia.org/wiki/George_Boole">George Boole</a>) operations:

In [ ]:
values = [False, False, False, False]
print( any(values) ) # Logical OR
print( all(values) ) # Logical AND

values = [False, False, True, False]
print( any(values) ) # Logical OR
print( all(values) ) # Logical AND

values = [True, True, True, True]
print( any(values) ) # Logical OR
print( all(values) ) # Logical AND

Pandas dataframes come with a similar <i>method</i> `.any()`. If we first call `.isna()` and then call `.any()` for the resulting dataframe, we obtain the following:

In [ ]:
minisharks.isna().any()

As you can see, the operation is a logical or but it operates on the <i>columns</i>, not the rows. As with many pandas methods, we can tell it to operate on rows instead using the `axis` argument: `axis=0` indicates that we want to operate on columns, `axis=1` that we want to operate on rows. So for our `minisharks` example, the following will result in a pandas `Series` (recall: `Series` are like `lists` with an index)  which contains `True` for every row that contains at least one NaN value and `False` for all others:

In [ ]:
minisharks.isna().any(axis=1)

Now put things together! We create a boolean `selection` which filters out rows which contain a NaN value and then use it as an index:

In [ ]:
selection = minisharks.isna().any(axis=1)
minisharks[selection]

We're almost there! The boolean index gives us all the rows for which `selection` is `True`, but we want those where it is `False`.
Pandas series have a succinct way of achieving this: the `~` operator will flip `True` and `False` values (this is called <i>logical negation</i> or <i>boolean negation</i>).

In [ ]:
selection = minisharks.isna().any(axis=1)
print(selection)
selection = ~selection # Negate array
print(selection)
minisharks[selection] 

Finally! We have only the one row left our example that did not contain any NaN values.

> Apply what you just learned to the *whole* `shark` dataset. Defining the selection is easily possible in a single line of code!

In [ ]:
selection = ... # TODO
sharks = sharks[selection] 

You can test whether you correctly removed all NaN values if 
the following cell does not throw an error:

In [ ]:
assert sharks.isna().any().any() == False
''.join([chr(ord(s)-1) for s in 'Hppe!kpc"'])

<div style="border: 4px solid #29afa0; padding: 10px; font-size: 1.4em; color: #4b3e4c; text-align: center">
    Before you continue, make sure that the above test passes,
    otherwise the following tasks will not be possible!
</div>

<h3>Cleaning up the `Year` column</h3>

Having removed all rows with NaN in it, we still are not done with preparing our dataset. There are additional issues that appear because our dataset contained these bad fields: pandas tries to guess a suitable datatype for each column, but this goes wrong if there is even a single bad field which does not fit with this type. We can query the type of a `Series` by using the `.dtype` member:

In [ ]:
sharks['Year'].dtype

`float64` means that the `Year` column contains floating point numbers (like the Python `float`). That is <b>not</b> a good representations for this type of data, we should use integers for, well, integral numbers. There are important but somewhat difficult to explain problems related to floating point numbers, suffices to say that they can exhibit some very unintuitive behaviour when used for comparisons and calculations. 

Changing the type of a series is done via the `.astype(...)` method.
This method will fail if pandas does not know how to conduct the conversion (we will see such a case later with the `Age` column).

In [ ]:
sharks['Year'] = sharks['Year'].astype(int)
sharks['Year'].head(5) # Check colum

As you can see, the type of the `Year` column is now `int64` (which for our purposes is the same as Python's `int`). 

There is yet another issue with the `Year` column related to the kinds of values that appear in it. For this purpose the `.unique()` method of pandas `Series` is very useful: it returns a list of the values that appear in the series, but each value exactly once (very similar to a `set`, but the return value is just a list). If we do that for the `Year` column, we get the following:

In [ ]:
sharks['Year'].unique()

Note that there is a value 0 there! It would be very surprising if we knew about shark attacks from that time.

We can use the related method `.value_counts()` which very helpfully tells us which values occur and how often (similar to what a Python `Counter` does!):

In [ ]:
sharks['Year'].value_counts().tail(10) # Only print a few

Let us query this `Series` for `0` to see how often that value
appears (and consequently how much work we will have to put in to fix this):

In [ ]:
sharks['Year'].value_counts()[0] # Check how often 0 appears

Well that's not too bad. Eight values means we can simply delete those rows without losing too many datapoints.

Let's apply what we learned about boolean indices to have a look at those rows. Pandas provides use with a very convenient shorthand that let's us compare every element of a `Series` to a single value. For example,

In [ ]:
(sharks['Year'] == 0).tail(10) # We use .tail(..) to avoid a really long output

Returns a list of boolean values, where an entry is `True` if `sharks['Year']` is equal to `0` and `False` otherwise. With this, we can easily filter out the rows that have a year-zero problem:

In [ ]:
selection = sharks['Year'] == 0
sharks[selection]

Clearly the value `0` was simply used to denote an unknown date. At other points, the value was simply missing instead---a sure sign that the dataset was created by humans in a spreadsheed (a proper database would not allow such mixed uses). 

Let us remove those few rows since we would like to work with a dataset in which every field has a reliable value.

> Create a boolean index which is <tt>True</tt> for all rows in which the year is <b>not</b> zero in the cell below and run it to
remove all 

In [ ]:
selection = ... # Select all rows in which the year is 0
sharks[selection] # Output for testing

# Only uncomment the line below if you are sure your code 
# is correct, otherwise you might have to re-run the earlier
# code to get `sharks` in its original state.
# sharks = sharks[selection]

In [ ]:
assert all(sharks['Year'] != 0), "Year zero in dataframe!"
''.join([chr(ord(s)-1) for s in 'Hppe!kpc"'])

<h2>Cleaning up the `age` column</h2>

If you thought the `Year` column was a mess, well, let me tell you, the `Age` column is so much worse. Let's see what type it is:

In [ ]:
sharks['Age'].dtype

A data type `dtype('O')` or `object` means that this column contains untyped data, meaning that pandas essentially gave up on casting it to any sensible value. Think of this as the column containing strings. Let's have a look at the values contain in the column:

In [ ]:
sharks['Age'].unique()

Remember what I wrote about humans and spreadsheets? This is what happens. We could try to untangle some of this mess (some rows simply seem to talk about multiple victims and we could maybe turns them into two rows by hand) but it would cost a lot of manual work. For our purposes it will be good enough to simply remove all rows in which we cannot turn the age into an integer.

This is surprisingly involved problem, mostly because `int` columns in `Series` cannot contain the value NaN (there is no symbol reserved for not-an-integer). There seems to be a support for this feature in newer pandas versions, but we will have to make due with a simpler approach.

First, the following `pandas` method converts a `Series` to numbers&mdash;integers, if all values in the series can be converted to integers, `float` otherwise with NaN for all non-convertible values.

In [ ]:
age_float = pd.to_numeric(sharks['Age'], errors='coerce')
age_float.loc[40:50] # There should be a NaN in this range

We certainly do not want to represent ages as floats! However, we can convert this series into a boolean index by asking which values are NaN, just as before:

In [ ]:
selection = age_float.isna()
selection.loc[40:50]

Finally, we can put the pieces together.

> Use the above selection to remove all rows from the `shark` dataset which can be converted to numbers. Afterwards, convert
the `age` column to integers using the method `.astype(...)`.

Hint: Check the section in which we cleaned up the `year` column to see how `.astype(...)` is used.

In [ ]:
# TODO! Clean up sharks['Age'] using the above `selection`

You can test whether you where successful by running the cell below.

In [ ]:
assert sharks['Age'].dtype == 'int64'
''.join([chr(ord(s)-1) for s in 'Hppe!kpc"'])

<h2>Saving your work</h2>

If all the above is done we should save all the hard work we put into cleaning up this dataset. Luckily, pandas makes it rather easy to write a file to .csv:

In [ ]:
sharks.to_csv('04-resources/sharks-cleaned.csv')